In [13]:
# Importing libraries
import pandas as pd
from sklearn.model_selection import train_test_split
#import the remaining librairies to be used
from sklearn.model_selection import GridSearchCV

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    recall_score,
    precision_score,
    classification_report,
)

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE


In [16]:
for i in df_final.columns:
    print(f"{i} -  {df_final[i].unique()}")

Age Group -  ['ADULT' 'CHILD' 'UNKNOWN']
Household Composition -  ['COHABITATES WITH OTHERS' 'LIVES ALONE' 'NOT APPLICABLE' 'UNKNOWN']
Special Education Services -  ['NOT APPLICABLE' 'YES' 'NO' 'UNKNOWN']
Mental Illness -  ['YES' 'NO']
No Chronic Med Condition -  ['YES' 'NO' 'UNKNOWN']
Smokes -  ['NO' 'YES' 'UNKNOWN']
Unknown Insurance Coverage -  ['NO' 'YES']
Criminal Justice Status -  ['NO' 'YES' 'UNKNOWN']
Program_Category -  ['Regular Treatment' 'Extra Help' 'Urgent Care']
Religion_Category -  ['Unknown' 'Formal Religion' 'Spiritual but not Religious']
Employment_Status -  ['Employed' 'Not in Labor Force' 'Unemployed' 'Unknown']
Hours_Category -  ['Part-Time' 'Full-Time' 'Unknown']
Education_Category -  ['Higher Education' 'Secondary Education' 'Unknown' 'Primary Education'
 'No Formal Education']
RACE -  ['WHITE' 'OTHER/MULTIRACIAL' 'BLACK' 'UNKNOWN']
hispanic_ethnicity -  ['HISPANIC' 'NON-HISPANIC' 'UNKNOWN']
Living_Situation -  ['PRIVATE RESIDENCE' 'OTHER' 'INSTITUTIONAL/UNKNOWN

In [14]:
# Reading dataset
df_final = pd.read_csv(r'final_data_for_model_building.csv'
                 ) 

df_final.head(5)

,Age Group,Household Composition,Special Education Services,Mental Illness,No Chronic Med Condition,Smokes,Unknown Insurance Coverage,Criminal Justice Status,Program_Category,Religion_Category,...,Heartchronic_Summary,Disorder_summary,Other_Chronic_Illness_Summmary,Brainchronic_Summary,Insured_or_Not,Has_Public_Insurance,Has_Private_or_Other_Insurance,Confirmed_Medicaid_Managed,Gender_Identity_Orientation,Receiving Cash Assistance
0,ADULT,COHABITATES WITH OTHERS,NOT APPLICABLE,YES,YES,NO,NO,NO,Regular Treatment,Unknown,...,"NO, HEART CHRONIC ILLNESS",NO DISORDER,"NO, CHRONIC ILLNESS","NO, BRAIN CHRONIC ILLNESS",Yes,Yes,No,Yes,Cisgender Man,No/Unknown
1,ADULT,LIVES ALONE,NOT APPLICABLE,YES,YES,NO,NO,NO,Regular Treatment,Formal Religion,...,"NO, HEART CHRONIC ILLNESS",NO DISORDER,"NO, CHRONIC ILLNESS","NO, BRAIN CHRONIC ILLNESS",Yes,Yes,No,Yes,Cisgender Man,No/Unknown
2,ADULT,COHABITATES WITH OTHERS,NOT APPLICABLE,YES,YES,YES,NO,NO,Regular Treatment,Formal Religion,...,"NO, HEART CHRONIC ILLNESS",ALCOHOL/DRUG DISORDER,"NO, CHRONIC ILLNESS","NO, BRAIN CHRONIC ILLNESS",Yes,Yes,No,Yes,Cisgender Man,No/Unknown
3,ADULT,NOT APPLICABLE,NOT APPLICABLE,YES,YES,YES,NO,NO,Regular Treatment,Unknown,...,"NO, HEART CHRONIC ILLNESS",ALCOHOL/DRUG DISORDER,"NO, CHRONIC ILLNESS","NO, BRAIN CHRONIC ILLNESS",Yes,Yes,No,No,Cisgender Man,Yes
4,ADULT,COHABITATES WITH OTHERS,NOT APPLICABLE,YES,NO,YES,NO,NO,Regular Treatment,Spiritual but not Religious,...,"NO, HEART CHRONIC ILLNESS",ALCOHOL/DRUG DISORDER,"NO, CHRONIC ILLNESS","NO, BRAIN CHRONIC ILLNESS",Yes,Yes,No,Yes,Cisgender Woman,Yes


In [3]:
df_final.columns

Index(['Age Group', 'Household Composition', 'Special Education Services',
       'Mental Illness', 'No Chronic Med Condition', 'Smokes',
       'Unknown Insurance Coverage', 'Criminal Justice Status',
       'Program_Category', 'Religion_Category', 'Employment_Status',
       'Hours_Category', 'Education_Category', 'RACE', 'hispanic_ethnicity',
       'Living_Situation', 'Diagnosis_Summary', 'Mental_Disability_Summary',
       'Impairment_Summary', 'Chronic_disease_Summary',
       'Canabis_Usage_Summary', 'Smoking treatment_summary',
       'Service_drug_alcohol_Summary', 'Other_testchronic_group_Summary',
       'Heartchronic_Summary', 'Disorder_summary',
       'Other_Chronic_Illness_Summmary', 'Brainchronic_Summary',
       'Insured_or_Not', 'Has_Public_Insurance',
       'Has_Private_or_Other_Insurance', 'Confirmed_Medicaid_Managed',
       'Gender_Identity_Orientation', 'Receiving Cash Assistance'],
      dtype='object')

In [4]:
for i in df_final.columns:
    print(i, df_final[i].nunique())

Age Group 3
Household Composition 4
Special Education Services 4
Mental Illness 2
No Chronic Med Condition 3
Smokes 3
Unknown Insurance Coverage 2
Criminal Justice Status 3
Program_Category 3
Religion_Category 3
Employment_Status 4
Hours_Category 3
Education_Category 5
RACE 4
hispanic_ethnicity 3
Living_Situation 3
Diagnosis_Summary 5
Mental_Disability_Summary 3
Impairment_Summary 3
Chronic_disease_Summary 2
Canabis_Usage_Summary 3
Smoking treatment_summary 3
Service_drug_alcohol_Summary 3
Other_testchronic_group_Summary 3
Heartchronic_Summary 3
Disorder_summary 3
Other_Chronic_Illness_Summmary 2
Brainchronic_Summary 3
Insured_or_Not 2
Has_Public_Insurance 2
Has_Private_or_Other_Insurance 2
Confirmed_Medicaid_Managed 2
Gender_Identity_Orientation 6
Receiving Cash Assistance 2


In [5]:
df_model = df_final.copy()

In [6]:
# 1) Label-encode Education_Category (ordered: low -> high)
#    We'll map "Unknown" to -1 to avoid implying a false order.
edu_order = {
    'No Formal Education': 0,
    'Primary Education': 1,
    'Secondary Education': 2,
    'Higher Education': 3,
    'Unknown': -1,  # keeping unknown separate; avoids misleading ordinal meaning
}

if 'Education_Category' in df_model.columns:
    df_model['Education_Category'] = (
        df_model['Education_Category'].astype(str).map(lambda x: x.strip()).map(edu_order).fillna(-1).astype(int)
    )


In [7]:
# encode a binary categorical column into a single 0/1 column
#    - Replaces the original column name with its dummy (drop_first=True).
#    - For labels like ['NO','YES'], the resulting column will be 1 for "YES".

def encode_binary_column(df, col):
    if col not in df.columns:
        return
    dummies = pd.get_dummies(df[col], drop_first=True, dtype=int)
    # If the column somehow isn't binary, skip safely
    if dummies.shape[1] != 1:
        return
    df.drop(columns=[col], inplace=True)
    df[col] = dummies.iloc[:, 0]

# Encode all required binary columns (2 categories)
# Target is 'Mental Illness' (Yes/No), that is also encoded here.

binary_cols = [
    'Mental Illness',
    'Unknown Insurance Coverage',
    'Insured_or_Not',
    'Receiving Cash Assistance',
    'Has_Public_Insurance',
    'Has_Private_or_Other_Insurance',
    'Confirmed_Medicaid_Managed',
    'Chronic_disease_Summary',
    'Other_Chronic_Illness_Summmary',
]

for col in binary_cols:
    encode_binary_column(df_model, col)

In [8]:

# df_model['mental_illness'] = df_model['Mental Illness']
# 5) One-hot encode all remaining multi-class categorical columns (3+ categories)
#    We’ll one-hot them in one shot. For tree models (RF/XGB), drop_first=False is fine.

multiclass_cols = [
    'Age Group',
    'Household Composition',
    'Special Education Services',
    'No Chronic Med Condition',
    'Smokes',
    'Criminal Justice Status',
    'Program_Category',
    'Religion_Category',
    'Employment_Status',
    'Hours_Category',
    'RACE',
    'hispanic_ethnicity',
    'Living_Situation',
    'Gender_Identity_Orientation',
    'Diagnosis_Summary',
    'Mental_Disability_Summary',
    'Impairment_Summary',
    'Canabis_Usage_Summary',
    'Smoking treatment_summary',
    'Service_drug_alcohol_Summary',
    'Other_testchronic_group_Summary',
    'Heartchronic_Summary',
    'Disorder_summary',
    'Brainchronic_Summary',
]

# Remove any columns that don’t exist or that are not object/categorical anymore
multiclass_cols = [c for c in multiclass_cols if c in df_model.columns]

# Identify which of these are still non-numeric (object/category) and need dummies
obj_to_dummy = [c for c in multiclass_cols if df_model[c].dtype == 'O']

if obj_to_dummy:
    dummies = pd.get_dummies(
        df_model[obj_to_dummy],
        prefix=obj_to_dummy,
        drop_first=False,
        dtype=int
    )
    df_model = pd.concat([df_model.drop(columns=obj_to_dummy), dummies], axis=1)

# 6) Final encoded frame ready for modeling
encoded_df = df_model.copy()
encoded_df.head()


,Education_Category,Mental Illness,Unknown Insurance Coverage,Insured_or_Not,Receiving Cash Assistance,Has_Public_Insurance,Has_Private_or_Other_Insurance,Confirmed_Medicaid_Managed,Chronic_disease_Summary,Other_Chronic_Illness_Summmary,...,"Other_testchronic_group_Summary_YES, HYPERLIPIDEMIA/HIGHBLOODPRESSURE/OBESITY","Heartchronic_Summary_NO, HEART CHRONIC ILLNESS",Heartchronic_Summary_UNKNOWN,"Heartchronic_Summary_YES, HEART CHRONIC ILLNESS",Disorder_summary_ALCOHOL/DRUG DISORDER,Disorder_summary_NO DISORDER,Disorder_summary_UNKNOWN,"Brainchronic_Summary_NO, BRAIN CHRONIC ILLNESS",Brainchronic_Summary_UNKNOWN,"Brainchronic_Summary_YES, BRAIN CHRONIC ILLNESS"
0,3,1,0,1,0,1,0,1,1,0,...,0,1,0,0,0,1,0,1,0,0
1,3,1,0,1,0,1,0,1,1,0,...,0,1,0,0,0,1,0,1,0,0
2,3,1,0,1,0,1,0,1,1,0,...,0,1,0,0,1,0,0,1,0,0
3,2,1,0,1,1,1,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
4,3,1,0,1,1,1,0,1,0,0,...,1,1,0,0,1,0,0,1,0,0


In [9]:
print(encoded_df.columns)

Index(['Education_Category', 'Mental Illness', 'Unknown Insurance Coverage',
       'Insured_or_Not', 'Receiving Cash Assistance', 'Has_Public_Insurance',
       'Has_Private_or_Other_Insurance', 'Confirmed_Medicaid_Managed',
       'Chronic_disease_Summary', 'Other_Chronic_Illness_Summmary',
       'Age Group_ADULT', 'Age Group_CHILD', 'Age Group_UNKNOWN',
       'Household Composition_COHABITATES WITH OTHERS',
       'Household Composition_LIVES ALONE',
       'Household Composition_NOT APPLICABLE', 'Household Composition_UNKNOWN',
       'Special Education Services_NO',
       'Special Education Services_NOT APPLICABLE',
       'Special Education Services_UNKNOWN', 'Special Education Services_YES',
       'No Chronic Med Condition_NO', 'No Chronic Med Condition_UNKNOWN',
       'No Chronic Med Condition_YES', 'Smokes_NO', 'Smokes_UNKNOWN',
       'Smokes_YES', 'Criminal Justice Status_NO',
       'Criminal Justice Status_UNKNOWN', 'Criminal Justice Status_YES',
       'Program_Categ

#### Model Building

In [10]:
# ================================================================
# Handling Class Imbalance in Classification
# ================================================================
# Problem:
#   Dataset is highly imbalanced → Baseline models predict majority class.
#
# Solution:
#   1. Use class_weight='balanced':
#       - Automatically adjusts weights inversely proportional to class frequencies.
#   2. Apply SMOTE (Synthetic Minority Over-sampling Technique):
#       - Creates synthetic samples for minority class.
#       - Applied only to training set (to prevent data leakage).
#
# Why Combine Both?
#   - SMOTE balances training data.
#   - class_weight ensures model still considers minority class importance.
#
# Expected Result:
#   - Higher recall and F1-score for minority class.
#   - Slight accuracy drop but fairer predictions.

In [ ]:
# Neural Network (MLPClassifier) Hyperparameter Tuning with GridSearchCV
# Goal:
#   - Enhance model performance by tuning neural network architecture and learning parameters.
#
# Why Important?
#   - Default MLP settings may underfit or overfit.
#   - Proper tuning of layers, neurons, and learning rate improves convergence and accuracy.
#
# Strategy:
#   - Use 3-fold Cross Validation for balanced evaluation vs. computation time.
#   - Optimize for F1-score to handle imbalanced classification.
#
# Parameters Tuned:
#   - hidden_layer_sizes: Number and size of hidden layers (controls network complexity).
#   - activation: Activation function (e.g., 'relu', 'tanh', 'logistic').
#   - solver: Optimization algorithm ('adam' or 'sgd').
#   - alpha: L2 regularization term (prevents overfitting).
#   - learning_rate_init: Initial learning rate for weight updates.
#
# Additional Handling:
#   - max_iter set high enough to ensure convergence.
#   - random_state fixed for reproducibility.
#   - early_stopping can be enabled for automatic convergence monitoring.


In [11]:
#  Prepare Data
df = encoded_df.copy()

# Define target and features
TARGET = 'Mental Illness'
X = df.drop(columns=[TARGET])
y = df[TARGET]

# Split into train and test sets (70% train, 30% test)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("Original class distribution:")
print(y.value_counts(normalize=True))

# -------------------------------
# 2. Apply SMOTE (ONLY on training set)
# -------------------------------
print("\nApplying SMOTE...")
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("After SMOTE class distribution:")
print(pd.Series(y_train_smote).value_counts())

# -------------------------------
# 3. Build Neural Network Model
# -------------------------------
mlp_model = MLPClassifier(
    hidden_layer_sizes=(100, 50),  # Two hidden layers
    activation='relu',             # ReLU activation
    solver='adam',                 # Optimizer
    max_iter=500,                  # Increase for convergence
    random_state=42
)

# Train the model
print("\nTraining Neural Network...")
mlp_model.fit(X_train_smote, y_train_smote)

# -------------------------------
# 4. Evaluate Model
# -------------------------------
y_pred_mlp = mlp_model.predict(X_test)

print("\n===== Neural Network (MLP) Results =====")
print(classification_report(y_test, y_pred_mlp))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_mlp))

Original class distribution:
Mental Illness
1    0.97127
0    0.02873
Name: proportion, dtype: float64

Applying SMOTE...
After SMOTE class distribution:
Mental Illness
1    131977
0    131977
Name: count, dtype: int64

Training Neural Network...

===== Neural Network (MLP) Results =====
              precision    recall  f1-score   support

           0       0.18      0.26      0.21      1673
           1       0.98      0.97      0.97     56563

    accuracy                           0.95     58236
   macro avg       0.58      0.61      0.59     58236
weighted avg       0.95      0.95      0.95     58236


Confusion Matrix:
 [[  427  1246]
 [ 1884 54679]]


In [12]:
import joblib

# Save model to file
joblib.dump(mlp_model, "best_mlp_model_model.pkl")
print("Model saved as best_mlp_model_model.pkl")


# Load model from file
loaded_model = joblib.load("best_mlp_model_model.pkl")

# Use loaded model to predict
y_pred_loaded = loaded_model.predict(X_test)
print("Predictions using loaded model:")
print(y_pred_loaded[:10])

Model saved as best_mlp_model_model.pkl
Predictions using loaded model:
[1 1 1 1 1 1 0 1 1 1]
